# Bollinger Bands - Complete Trading Tutorial

## Overview
This notebook provides a comprehensive tutorial on Bollinger Bands, including:
- Concept and theory
- Mathematical formula and calculation
- Manual implementation in Python
- TradingView integration
- Binance platform usage
- Practical trading strategies
- Backtesting examples

## 1. Introduction to Bollinger Bands

### What are Bollinger Bands?
Bollinger Bands are a volatility-based technical analysis indicator developed by John Bollinger in the 1980s. They consist of three lines: a simple moving average (middle band), an upper band, and a lower band. The upper and lower bands are typically positioned two standard deviations away from the middle band.

### Why Use Bollinger Bands?
- **Volatility Measurement**: Bands widen during high volatility and contract during low volatility
- **Overbought/Oversold**: Price touching upper/lower bands can indicate extreme conditions
- **Trend Identification**: Price action relative to bands shows trend strength
- **Signal Generation**: Band squeezes often precede significant price moves

### Best Timeframes and Markets
- **Short-term**: Bollinger Bands(10, 2) for day trading
- **Medium-term**: Bollinger Bands(20, 2) for swing trading
- **Long-term**: Bollinger Bands(50, 2) for position trading
- **Markets**: Works well in both trending and ranging markets

## 2. Mathematical Formula

### Bollinger Bands Formula
```
Middle Band = SMA(n)
Upper Band = SMA(n) + (k × σ)
Lower Band = SMA(n) - (k × σ)

Where:
- SMA(n) = Simple Moving Average over n periods
- σ = Standard deviation over n periods
- k = Number of standard deviations (typically 2)
- n = Number of periods (typically 20)
```

### Parameters:
- **Period (n)**: Number of periods for SMA and standard deviation (typically 20)
- **Standard Deviations (k)**: Number of standard deviations for bands (typically 2)
- **%B**: Measures where price is relative to the bands (0-100 scale)
- **Bandwidth**: Measures the width of the bands relative to the middle band

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import sys
sys.path.append('../utils')

# Import utilities
from data_downloader import DataDownloader
from indicators import IndicatorCalculator
from backtest_engine import BacktestEngine, StrategyGenerator

# Set style for better visualizations
plt.style.use('seaborn-v0-8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

## 3. Data Download

Let's download historical crypto data for our analysis.

In [ ]:
# Initialize data downloader
downloader = DataDownloader()

# Download BTC/USDT data
pair = "BTC/USDT"
timeframe = "1d"
days = 365

print(f"Downloading {pair} data for {days} days...")
df = downloader.get_market_data(pair, timeframe, days, source='ccxt')

if df is not None:
    print(f"\nDownloaded {len(df)} candles of {pair} data")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    print(f"\nFirst few rows:")
    display(df.head())
else:
    print("Failed to download data")

## 4. Manual Calculation of Bollinger Bands

Let's implement Bollinger Bands from scratch to understand how they work.

In [ ]:
def calculate_bollinger_bands_manual(data, period=20, std_dev=2):
    """
    Calculate Bollinger Bands manually
    """
    df = data.copy()
    
    # Calculate middle band (SMA)
    df['middle_band'] = df['close'].rolling(window=period).mean()
    
    # Calculate standard deviation
    df['std_dev'] = df['close'].rolling(window=period).std()
    
    # Calculate upper and lower bands
    df['upper_band'] = df['middle_band'] + (df['std_dev'] * std_dev)
    df['lower_band'] = df['middle_band'] - (df['std_dev'] * std_dev)
    
    # Calculate %B
    df['percent_b'] = (df['close'] - df['lower_band']) / (df['upper_band'] - df['lower_band'])
    
    # Calculate Bandwidth
    df['bandwidth'] = (df['upper_band'] - df['lower_band']) / df['middle_band']
    
    return df[['middle_band', 'upper_band', 'lower_band', 'percent_b', 'bandwidth']]

# Calculate Bollinger Bands manually
df_manual = df.copy()
bb_manual = calculate_bollinger_bands_manual(df_manual, period=20, std_dev=2)
df_manual['middle_band'] = bb_manual['middle_band']
df_manual['upper_band'] = bb_manual['upper_band']
df_manual['lower_band'] = bb_manual['lower_band']
df_manual['percent_b'] = bb_manual['percent_b']
df_manual['bandwidth'] = bb_manual['bandwidth']

print("Bollinger Bands calculated manually:")
display(df_manual[['close', 'middle_band', 'upper_band', 'lower_band', 'percent_b', 'bandwidth']].tail(10))

# Let's verify the calculation manually
print("\nManual verification of Bollinger Bands for the last row:")
recent_data = df_manual['close'].tail(20)
manual_sma = recent_data.mean()
manual_std = recent_data.std()
manual_upper = manual_sma + (manual_std * 2)
manual_lower = manual_sma - (manual_std * 2)
manual_percent_b = (df_manual['close'].iloc[-1] - manual_lower) / (manual_upper - manual_lower)

print(f"20-period SMA: {manual_sma:.2f}")
print(f"20-period Std Dev: {manual_std:.2f}")
print(f"Manual Upper Band: {manual_upper:.2f}")
print(f"Calculated Upper Band: {df_manual['upper_band'].iloc[-1]:.2f}")
print(f"Manual Lower Band: {manual_lower:.2f}")
print(f"Calculated Lower Band: {df_manual['lower_band'].iloc[-1]:.2f}")
print(f"Manual %B: {manual_percent_b:.4f}")
print(f"Calculated %B: {df_manual['percent_b'].iloc[-1]:.4f}")

## 5. Library Implementation

Now let's use established libraries to calculate Bollinger Bands.

In [ ]:
# Using our utilities
df_utils = df.copy()
df_utils['middle_band_utils'], df_utils['upper_band_utils'], df_utils['lower_band_utils'] = IndicatorCalculator.bollinger_bands(
    df_utils['close'], period=20, std_dev=2
)

# Using pandas-ta
import pandas_ta as ta
df_ta = df.copy()
df_ta.ta.bbands(length=20, std=2, append=True)

# Using TA-Lib
try:
    import talib
    df_talib = df.copy()
    df_talib['upper_band_talib'], df_talib['middle_band_talib'], df_talib['lower_band_talib'] = talib.BBANDS(
        df_talib['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    print("TA-Lib Bollinger Bands calculated successfully")
except ImportError:
    print("TA-Lib not installed, skipping TA-Lib example")
    df_talib = df.copy()
    df_talib['upper_band_talib'] = np.nan
    df_talib['middle_band_talib'] = np.nan
    df_talib['lower_band_talib'] = np.nan

# Compare results
print("Comparison of different Bollinger Bands implementations:")
comparison = pd.DataFrame({
    'Manual_Upper': df_manual['upper_band'],
    'Manual_Middle': df_manual['middle_band'],
    'Manual_Lower': df_manual['lower_band'],
    'Utils_Upper': df_utils['upper_band_utils'],
    'Utils_Middle': df_utils['middle_band_utils'],
    'Utils_Lower': df_utils['lower_band_utils'],
    'Pandas-TA_Upper': df_ta['BBU_20_2.0'],
    'Pandas-TA_Middle': df_ta['BBM_20_2.0'],
    'Pandas-TA_Lower': df_ta['BBL_20_2.0'],
    'TA-Lib_Upper': df_talib['upper_band_talib'],
    'TA-Lib_Middle': df_talib['middle_band_talib'],
    'TA-Lib_Lower': df_talib['lower_band_talib']
})

display(comparison.dropna().tail(10))

# Check if all methods produce similar results
print("\nDifferences between methods:")
print(f"Manual vs Utils Upper max difference: {abs(comparison['Manual_Upper'] - comparison['Utils_Upper']).max():.8f}")
print(f"Manual vs Utils Middle max difference: {abs(comparison['Manual_Middle'] - comparison['Utils_Middle']).max():.8f}")
print(f"Manual vs Pandas-TA Upper max difference: {abs(comparison['Manual_Upper'] - comparison['Pandas-TA_Upper']).max():.8f}")

## 6. Visualization

Let's visualize Bollinger Bands with price action.

In [ ]:
# Create interactive plot
fig = go.Figure()

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_manual.index,
        open=df_manual['open'],
        high=df_manual['high'],
        low=df_manual['low'],
        close=df_manual['close'],
        name='Price'
    )
)

# Bollinger Bands
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['upper_band'],
        mode='lines',
        name='Upper Band',
        line=dict(color='red', width=2)
    )
)

fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['middle_band'],
        mode='lines',
        name='Middle Band',
        line=dict(color='blue', width=2)
    )
)

fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['lower_band'],
        mode='lines',
        name='Lower Band',
        line=dict(color='green', width=2)
    )
)

# Fill between bands
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['upper_band'],
        mode='lines',
        line=dict(width=0),
        showlegend=False
    )
)

fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['lower_band'],
        mode='lines',
        line=dict(width=0),
        fillcolor='rgba(128, 128, 128, 0.1)',
        fill='tonexty',
        name='Bands Area',
        showlegend=False
    )
)

fig.update_layout(
    title=f'{pair} - Bollinger Bands (20, 2)',
    template='plotly_dark',
    height=600,
    xaxis_rangeslider_visible=False
)

fig.show()

# Create static plot showing Bollinger Bands characteristics
plt.figure(figsize=(15, 10))

# Price and Bollinger Bands
plt.subplot(2, 1, 1)
plt.plot(df_manual.index, df_manual['close'], label='Price', alpha=0.7, color='black')
plt.plot(df_manual.index, df_manual['upper_band'], label='Upper Band', alpha=0.8, color='red')
plt.plot(df_manual.index, df_manual['middle_band'], label='Middle Band', alpha=0.8, color='blue')
plt.plot(df_manual.index, df_manual['lower_band'], label='Lower Band', alpha=0.8, color='green')
plt.fill_between(df_manual.index, df_manual['upper_band'], 
                df_manual['lower_band'], alpha=0.1, color='gray')
plt.title(f'{pair} - Bollinger Bands (20, 2)')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

# %B and Bandwidth
plt.subplot(2, 1, 2)
plt.plot(df_manual.index, df_manual['percent_b'], label='%B', alpha=0.8, color='purple')
plt.plot(df_manual.index, df_manual['bandwidth'], label='Bandwidth', alpha=0.8, color='orange')
plt.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='%B Upper (1)')
plt.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='%B Lower (0)')
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='%B Middle (0.5)')
plt.title('%B and Bandwidth')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. TradingView Integration

### Pine Script for Bollinger Bands

```pine
// Bollinger Bands Pine Script
//@version=5
indicator("Bollinger Bands", shorttitle="BB", format=format.price, precision=2)

// Input parameters
length = input.int(20, title="Length", minval=1)
mult = input.float(2.0, title="Standard Deviations", minval=0.1, step=0.1)
src = input.source(close, "Source")

// Calculate Bollinger Bands
basis = ta.sma(src, length)
dev = mult * ta.stdev(src, length)
upper = basis + dev
lower = basis - dev

// Plot Bollinger Bands
plot(basis, "Basis", color=color.new(color.blue, 0))
p1 = plot(upper, "Upper Band", color=color.new(color.red, 0))
p2 = plot(lower, "Lower Band", color=color.new(color.green, 0))
fill(p1, p2, color=color.new(color.gray, 90), title="Band Fill")

// Calculate and plot %B
percentB = (src - lower) / (upper - lower)
plot(percentB, "%B", color=color.new(color.purple, 0))
hline(1, "Overbought", color=color.new(color.red, 50))
hline(0, "Oversold", color=color.new(color.green, 50))
hline(0.5, "Midline", color=color.new(color.gray, 50))

// Add signals
buySignal = ta.crossunder(src, lower)
sellSignal = ta.crossover(src, upper)

plotshape(buySignal, title="Buy Signal", location=location.belowbar, color=color.green, style=shape.triangleup, size=size.small)
plotshape(sellSignal, title="Sell Signal", location=location.abovebar, color=color.red, style=shape.triangledown, size=size.small)

// Bandwidth calculation
bandwidth = (upper - lower) / basis * 100
plot(bandwidth, "Bandwidth", color=color.new(color.orange, 0), style=plot.style_area)
```

### How to Add to TradingView:
1. Open TradingView chart
2. Click on "Pine Editor" tab at the bottom
3. Copy and paste the script above
4. Click "Add to Chart"
5. Adjust parameters in the settings

### TradingView Built-in Bollinger Bands:
1. Click on "Indicators" at the top
2. Search for "Bollinger Bands"
3. Select "Bollinger Bands" from the list
4. Adjust length and standard deviation parameters

## 8. Binance Integration

### How to Add Bollinger Bands on Binance:
1. Open Binance trading interface
2. Select your trading pair (e.g., BTC/USDT)
3. Click on "Indicators" at the top of the chart
4. Search for "Bollinger Bands"
5. Select "Bollinger Bands" from the list
6. In the settings:
   - Set "Length" (default: 20)
   - Set "StdDev" (default: 2)
   - Choose colors and line styles
7. Click "Apply"

### Binance Bollinger Bands Parameters:
- **Length**: 1-500 (common values: 10, 20, 50)
- **StdDev**: 0.1-5.0 (common values: 1.5, 2.0, 2.5)
- **Source**: Close, Open, High, Low, HL/2, HLC/3, OHLC/4
- **Colors**: Custom color selection for upper, middle, and lower bands
- **Line Style**: Solid, Dashed, Dotted
- **Line Width**: 1-5 pixels

## 9. Trading Strategies

### Strategy 1: Bollinger Bands Mean Reversion
**Description**: Buy when price touches lower band, sell when price touches upper band

**Rules**:
- **Entry**: Price closes below lower band
- **Exit**: Price closes above upper band
- **Stop Loss**: 2-3% below entry price
- **Take Profit**: 5-10% above entry price or when opposite signal occurs

In [ ]:
def bb_mean_reversion_strategy(data, period=20, std_dev=2):
    """
    Bollinger Bands Mean Reversion Strategy
    """
    df = data.copy()
    
    # Calculate Bollinger Bands
    df['middle_band'], df['upper_band'], df['lower_band'] = IndicatorCalculator.bollinger_bands(
        df['close'], period, std_dev
    )
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['close'] <= df['lower_band'], 'signal'] = 1  # Buy when below lower band
    df.loc[df['close'] >= df['upper_band'], 'signal'] = -1  # Sell when above upper band
    
    # Identify band touches
    df['lower_band_touch'] = ((df['close'] <= df['lower_band']) & 
                               (df['close'].shift(1) > df['lower_band'].shift(1)))
    df['upper_band_touch'] = ((df['close'] >= df['upper_band']) & 
                               (df['close'].shift(1) < df['upper_band'].shift(1)))
    
    return df

# Apply strategy
df_bb_mr = bb_mean_reversion_strategy(df_manual)

# Plot signals
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'{pair} - Bollinger Bands Mean Reversion Strategy', f'{pair} - %B and Bandwidth'),
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3]
)

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_bb_mr.index,
        open=df_bb_mr['open'],
        high=df_bb_mr['high'],
        low=df_bb_mr['low'],
        close=df_bb_mr['close'],
        name='Price'
    ),
    row=1, col=1
)

# Bollinger Bands
fig.add_trace(
    go.Scatter(
        x=df_bb_mr.index,
        y=df_bb_mr['upper_band'],
        mode='lines',
        name='Upper Band',
        line=dict(color='red', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_bb_mr.index,
        y=df_bb_mr['middle_band'],
        mode='lines',
        name='Middle Band',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_bb_mr.index,
        y=df_bb_mr['lower_band'],
        mode='lines',
        name='Lower Band',
        line=dict(color='green', width=2)
    ),
    row=1, col=1
)

# Buy signals
buy_signals = df_bb_mr[df_bb_mr['lower_band_touch']]
fig.add_trace(
    go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-up', size=12, color='green'),
        name='Buy Signal'
    ),
    row=1, col=1
)

# Sell signals
sell_signals = df_bb_mr[df_bb_mr['upper_band_touch']]
fig.add_trace(
    go.Scatter(
        x=sell_signals.index,
        y=sell_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-down', size=12, color='red'),
        name='Sell Signal'
    ),
    row=1, col=1
)

# %B and Bandwidth
fig.add_trace(
    go.Scatter(
        x=df_bb_mr.index,
        y=df_bb_mr['percent_b'],
        mode='lines',
        name='%B',
        line=dict(color='purple', width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_bb_mr.index,
        y=df_bb_mr['bandwidth'],
        mode='lines',
        name='Bandwidth',
        line=dict(color='orange', width=2)
    ),
    row=2, col=1
)

# %B levels
fig.add_hline(y=1, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_hline(y=0, line=dict(color='green', dash='dash'), row=2, col=1)
fig.add_hline(y=0.5, line=dict(color='gray', dash='dot'), row=2, col=1)

fig.update_layout(
    title=f'{pair} - Bollinger Bands Mean Reversion Strategy',
    template='plotly_dark',
    height=800
)

fig.show()

print(f"Total buy signals: {len(buy_signals)}")
print(f"Total sell signals: {len(sell_signals)}")
print(f"\nRecent signals:")
recent_signals = df_bb_mr[df_bb_mr['lower_band_touch'] | df_bb_mr['upper_band_touch']].tail(10)
display(recent_signals[['close', 'percent_b', 'lower_band_touch', 'upper_band_touch']])

### Strategy 2: Bollinger Bands Squeeze
**Description**: Trade the expansion after a period of low volatility (band squeeze)

**Rules**:
- **Setup**: Bandwidth reaches low levels (indicating low volatility)
- **Entry**: Price breaks out of the squeeze with increasing volume
- **Exit**: When bandwidth expands significantly or price reaches opposite band
- **Stop Loss**: Below/above the breakout point
- **Take Profit**: Multiple of the breakout range

In [ ]:
def bb_squeeze_strategy(data, period=20, std_dev=2, bandwidth_threshold=0.1):
    """
    Bollinger Bands Squeeze Strategy
    """
    df = data.copy()
    
    # Calculate Bollinger Bands
    df['middle_band'], df['upper_band'], df['lower_band'] = IndicatorCalculator.bollinger_bands(
        df['close'], period, std_dev
    )
    
    # Calculate %B and Bandwidth
    df['percent_b'] = (df['close'] - df['lower_band']) / (df['upper_band'] - df['lower_band'])
    df['bandwidth'] = (df['upper_band'] - df['lower_band']) / df['middle_band']
    
    # Identify squeeze (low bandwidth)
    df['squeeze'] = df['bandwidth'] < bandwidth_threshold
    
    # Identify breakout from squeeze
    df['breakout_up'] = df['squeeze'].shift(1) & (df['close'] > df['upper_band']) & (df['bandwidth'] > df['bandwidth'].shift(1))
    df['breakout_down'] = df['squeeze'].shift(1) & (df['close'] < df['lower_band']) & (df['bandwidth'] > df['bandwidth'].shift(1))
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['breakout_up'], 'signal'] = 1
    df.loc[df['breakout_down'], 'signal'] = -1
    
    return df

# Apply strategy
df_bb_squeeze = bb_squeeze_strategy(df_manual)

# Create signals DataFrame for backtesting
bb_squeeze_signals = pd.DataFrame(index=df_bb_squeeze.index)
bb_squeeze_signals['signal'] = df_bb_squeeze['signal']

print(f"Squeeze periods identified: {df_bb_squeeze['squeeze'].sum()}")
print(f"Upward breakouts: {df_bb_squeeze['breakout_up'].sum()}")
print(f"Downward breakouts: {df_bb_squeeze['breakout_down'].sum()}")

# Plot squeeze strategy
plt.figure(figsize=(15, 12))

plt.subplot(3, 1, 1)
plt.plot(df_bb_squeeze.index, df_bb_squeeze['close'], label='Price', alpha=0.7, color='black')
plt.plot(df_bb_squeeze.index, df_bb_squeeze['upper_band'], label='Upper Band', alpha=0.8, color='red')
plt.plot(df_bb_squeeze.index, df_bb_squeeze['middle_band'], label='Middle Band', alpha=0.8, color='blue')
plt.plot(df_bb_squeeze.index, df_bb_squeeze['lower_band'], label='Lower Band', alpha=0.8, color='green')
plt.fill_between(df_bb_squeeze.index, df_bb_squeeze['upper_band'], 
                df_bb_squeeze['lower_band'], alpha=0.1, color='gray')

# Highlight squeeze periods
squeeze_periods = df_bb_squeeze[df_bb_squeeze['squeeze']]
if len(squeeze_periods) > 0:
    for i, (date, row) in enumerate(squeeze_periods.iterrows()):
        if i % 5 == 0:  # Show every 5th squeeze period to avoid clutter
            plt.axvspan(date, date + pd.Timedelta(days=5), alpha=0.2, color='yellow', label='Squeeze' if i == 0 else '')

# Mark breakouts
plt.scatter(df_bb_squeeze[df_bb_squeeze['breakout_up']].index, 
            df_bb_squeeze[df_bb_squeeze['breakout_up']]['close'], 
            color='green', s=100, marker='^', label='Breakout Up', zorder=5)
plt.scatter(df_bb_squeeze[df_bb_squeeze['breakout_down']].index, 
            df_bb_squeeze[df_bb_squeeze['breakout_down']]['close'], 
            color='red', s=100, marker='v', label='Breakout Down', zorder=5)

plt.title(f'{pair} - Bollinger Bands Squeeze Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(df_bb_squeeze.index, df_bb_squeeze['bandwidth'], label='Bandwidth', alpha=0.8, color='orange')
plt.axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='Squeeze Threshold')
plt.fill_between(df_bb_squeeze.index, df_bb_squeeze['bandwidth'], 0.1, alpha=0.3, color='orange')
plt.title('Bandwidth')
plt.ylabel('Bandwidth')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 3)
plt.plot(df_bb_squeeze.index, df_bb_squeeze['percent_b'], label='%B', alpha=0.8, color='purple')
plt.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='%B Upper (1)')
plt.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='%B Lower (0)')
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='%B Middle (0.5)')
plt.title('%B')
plt.xlabel('Date')
plt.ylabel('%B')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Strategy 3: Bollinger Bands Trend Following
**Description**: Use %B to identify and follow trends

**Rules**:
- **Uptrend**: %B consistently above 0.5
- **Downtrend**: %B consistently below 0.5
- **Entry**: When %B confirms trend direction
- **Exit**: When %B crosses 0.5 in opposite direction

In [ ]:
def bb_trend_following_strategy(data, period=20, std_dev=2):
    """
    Bollinger Bands Trend Following Strategy
    """
    df = data.copy()
    
    # Calculate Bollinger Bands
    df['middle_band'], df['upper_band'], df['lower_band'] = IndicatorCalculator.bollinger_bands(
        df['close'], period, std_dev
    )
    
    # Calculate %B
    df['percent_b'] = (df['close'] - df['lower_band']) / (df['upper_band'] - df['lower_band'])
    
    # Generate signals based on %B
    df['signal'] = 0
    df.loc[df['percent_b'] > 0.5, 'signal'] = 1  # Long position
    df.loc[df['percent_b'] < 0.5, 'signal'] = -1  # Short position
    
    # Identify trend changes
    df['trend_up'] = (df['percent_b'] > 0.5) & (df['percent_b'].shift(1) <= 0.5)
    df['trend_down'] = (df['percent_b'] < 0.5) & (df['percent_b'].shift(1) >= 0.5)
    
    return df

# Apply strategy
df_bb_trend = bb_trend_following_strategy(df_manual)

# Create signals DataFrame for backtesting
bb_trend_signals = pd.DataFrame(index=df_bb_trend.index)
bb_trend_signals['signal'] = df_bb_trend['signal']

print(f"Trend up signals: {df_bb_trend['trend_up'].sum()}")
print(f"Trend down signals: {df_bb_trend['trend_down'].sum()}")

# Plot trend following strategy
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(df_bb_trend.index, df_bb_trend['close'], label='Price', alpha=0.7, color='black')
plt.plot(df_bb_trend.index, df_bb_trend['upper_band'], label='Upper Band', alpha=0.8, color='red')
plt.plot(df_bb_trend.index, df_bb_trend['middle_band'], label='Middle Band', alpha=0.8, color='blue')
plt.plot(df_bb_trend.index, df_bb_trend['lower_band'], label='Lower Band', alpha=0.8, color='green')
plt.fill_between(df_bb_trend.index, df_bb_trend['upper_band'], 
                df_bb_trend['lower_band'], alpha=0.1, color='gray')

# Color background based on trend
plt.fill_between(df_bb_trend.index, df_bb_trend['upper_band'], df_bb_trend['lower_band'], 
                where=(df_bb_trend['percent_b'] > 0.5), alpha=0.1, color='green', label='Uptrend')
plt.fill_between(df_bb_trend.index, df_bb_trend['upper_band'], df_bb_trend['lower_band'], 
                where=(df_bb_trend['percent_b'] < 0.5), alpha=0.1, color='red', label='Downtrend')

# Mark trend changes
plt.scatter(df_bb_trend[df_bb_trend['trend_up']].index, 
            df_bb_trend[df_bb_trend['trend_up']]['close'], 
            color='green', s=100, marker='^', label='Trend Up', zorder=5)
plt.scatter(df_bb_trend[df_bb_trend['trend_down']].index, 
            df_bb_trend[df_bb_trend['trend_down']]['close'], 
            color='red', s=100, marker='v', label='Trend Down', zorder=5)

plt.title(f'{pair} - Bollinger Bands Trend Following Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_bb_trend.index, df_bb_trend['percent_b'], label='%B', alpha=0.8, color='purple')
plt.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='%B Upper (1)')
plt.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='%B Lower (0)')
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='%B Middle (0.5)')
plt.fill_between(df_bb_trend.index, df_bb_trend['percent_b'], 0.5, alpha=0.3, color='green')
plt.fill_between(df_bb_trend.index, df_bb_trend['percent_b'], 0.5, alpha=0.3, color='red')
plt.title('%B')
plt.xlabel('Date')
plt.ylabel('%B')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Backtesting Results

Let's backtest our strategies and analyze the results.

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(initial_capital=10000)

# Create signals for each strategy
strategies = {
    'BB_Mean_Reversion': df_bb_mr[['signal']],
    'BB_Squeeze': bb_squeeze_signals,
    'BB_Trend_Following': bb_trend_signals
}

# Run backtests
for name, signals in strategies.items():
    print(f"Backtesting {name} strategy...")
    engine.run_backtest(df_manual, signals, name)

# Compare strategies
comparison = engine.compare_strategies(list(strategies.keys()))
print("\nStrategy Comparison:")
display(comparison)

# Plot equity curves
engine.plot_equity_curve()

# Plot drawdowns
engine.plot_drawdown()

In [ ]:
# Generate detailed report for the best strategy
best_strategy = comparison['total_return'].idxmax()
print(f"Best performing strategy: {best_strategy}")
print(engine.generate_report(best_strategy))

In [ ]:
# Analyze trade statistics for the best strategy
best_data = engine.results[best_strategy]['data']

# Analyze Bollinger Bands levels at trade entries
trade_analysis = best_data[best_data['signal'] != 0].copy()
trade_analysis['percent_b'] = df_manual['percent_b']
trade_analysis['bandwidth'] = df_manual['bandwidth']
trade_analysis['upper_band'] = df_manual['upper_band']
trade_analysis['lower_band'] = df_manual['lower_band']

if len(trade_analysis) > 0:
    print(f"\nTrade Analysis for {best_strategy}:")
    print(f"Total trades: {len(trade_analysis)}")
    print(f"Buy trades: {len(trade_analysis[trade_analysis['signal'] == 1])}")
    print(f"Sell trades: {len(trade_analysis[trade_analysis['signal'] == -1])}")
    print(f"Average %B at entry: {trade_analysis['percent_b'].mean():.3f}")
    print(f"Average Bandwidth at entry: {trade_analysis['bandwidth'].mean():.3f}")
    print(f"%B range at entry: {trade_analysis['percent_b'].min():.3f} - {trade_analysis['percent_b'].max():.3f}")
    
    # %B distribution at trade entries
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(trade_analysis['percent_b'], bins=20, alpha=0.7, edgecolor='black')
    plt.axvline(x=0, color='green', linestyle='--', label='%B Lower (0)')
    plt.axvline(x=1, color='red', linestyle='--', label='%B Upper (1)')
    plt.axvline(x=0.5, color='gray', linestyle=':', label='%B Middle (0.5)')
    plt.title('%B Distribution at Trade Entries')
    plt.xlabel('%B')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Performance by %B zone
    trade_analysis['pb_zone'] = pd.cut(trade_analysis['percent_b'], 
                                    bins=[-0.5, 0, 0.5, 1, 1.5], 
                                    labels=['Below Lower', 'Lower-Middle', 'Middle-Upper', 'Above Upper'])
    
    plt.subplot(1, 2, 2)
    zone_performance = trade_analysis.groupby('pb_zone')['strategy_returns'].mean() * 100
    zone_performance.plot(kind='bar', color=['green', 'blue', 'orange', 'red'])
    plt.title('Average Returns by %B Zone')
    plt.xlabel('%B Zone')
    plt.ylabel('Average Return (%)')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades completed in the backtest period")

## 11. Pros and Cons

### Advantages:
- ✅ **Volatility Adaptive**: Bands automatically adjust to market volatility
- ✅ **Overbought/Oversold**: Clear levels for extreme conditions
- ✅ **Trend Identification**: Price action relative to bands shows trend strength
- ✅ **Squeeze Detection**: Bandwidth can identify low volatility periods
- ✅ **Versatile**: Works in various market conditions

### Disadvantages:
- ❌ **False Signals**: Can give false signals in strong trending markets
- ❌ **Lagging**: Based on historical price data
- ❌ **Parameter Sensitivity**: Results can vary with different parameters
- ❌ **Whipsaw**: Can generate false signals in choppy markets

### When to Use:
- **Ranging Markets**: Excellent for sideways price action
- **Breakout Trading**: Good for identifying squeeze breakouts
- **Mean Reversion**: Effective for price returning to the mean
- **Volatility Analysis**: Useful for measuring market volatility

### When to Avoid:
- **Strong Trends**: Can give false signals in trending markets
- **Low Volatility**: Less reliable in very low volatility periods
- **As Standalone**: Should not be used as the only indicator
- **News Events**: Can be unreliable during high volatility

## 12. Advanced Bollinger Bands Techniques

### 1. Multiple Bollinger Bands
Use multiple sets of Bollinger Bands with different parameters:
- **Short-term**: BB(10, 1.5) for entry timing
- **Medium-term**: BB(20, 2) for trend identification
- **Long-term**: BB(50, 2.5) for major trend direction

### 2. Bollinger Bands with Other Indicators
Combine Bollinger Bands with other indicators for confirmation:
- **RSI Confirmation**: Only take signals when RSI confirms the direction
- **Volume Confirmation**: Look for volume to confirm breakouts
- **MACD Confirmation**: Use MACD to confirm trend strength

### 3. Bollinger Band Walks
Identify when price "walks" up or down the bands:
- **Upper Band Walk**: Price consistently touching upper band indicates strong uptrend
- **Lower Band Walk**: Price consistently touching lower band indicates strong downtrend

In [ ]:
# Advanced Bollinger Bands Techniques
df_advanced = df_manual.copy()

# 1. Multiple Bollinger Bands
df_advanced['bb_10_1.5_middle'], df_advanced['bb_10_1.5_upper'], df_advanced['bb_10_1.5_lower'] = IndicatorCalculator.bollinger_bands(
    df_advanced['close'], period=10, std_dev=1.5
)
df_advanced['bb_50_2.5_middle'], df_advanced['bb_50_2.5_upper'], df_advanced['bb_50_2.5_lower'] = IndicatorCalculator.bollinger_bands(
    df_advanced['close'], period=50, std_dev=2.5
)

# 2. Bollinger Band Walks
df_advanced['upper_band_walk'] = (df_advanced['close'] > df_advanced['upper_band']).rolling(window=5).sum() >= 3
df_advanced['lower_band_walk'] = (df_advanced['close'] < df_advanced['lower_band']).rolling(window=5).sum() >= 3

# 3. Bandwidth Rate of Change
df_advanced['bandwidth_roc'] = df_advanced['bandwidth'].pct_change() * 100

# 4. %B Divergence
df_advanced['price_pb_corr'] = df_advanced['close'].rolling(window=20).corr(df_advanced['percent_b'])

# Plot advanced techniques
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Multiple Bollinger Bands
axes[0, 0].plot(df_advanced.index, df_advanced['close'], label='Price', alpha=0.7, color='black')
axes[0, 0].plot(df_advanced.index, df_advanced['upper_band'], label='BB(20,2) Upper', alpha=0.8, color='red')
axes[0, 0].plot(df_advanced.index, df_advanced['lower_band'], label='BB(20,2) Lower', alpha=0.8, color='green')
axes[0, 0].plot(df_advanced.index, df_advanced['bb_10_1.5_upper'], label='BB(10,1.5) Upper', alpha=0.6, color='orange')
axes[0, 0].plot(df_advanced.index, df_advanced['bb_10_1.5_lower'], label='BB(10,1.5) Lower', alpha=0.6, color='blue')
axes[0, 0].set_title('Multiple Bollinger Bands')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Bandwidth Analysis
axes[0, 1].plot(df_advanced.index, df_advanced['bandwidth'], label='Bandwidth', color='orange')
axes[0, 1].plot(df_advanced.index, df_advanced['bandwidth_roc'], label='Bandwidth ROC', color='purple')
axes[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0, 1].fill_between(df_advanced.index, df_advanced['bandwidth_roc'], 0, alpha=0.3, color='purple')
axes[0, 1].set_title('Bandwidth and Rate of Change')
axes[0, 1].set_ylabel('Bandwidth / ROC (%)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Bollinger Band Walks
axes[1, 0].plot(df_advanced.index, df_advanced['close'], label='Price', alpha=0.7, color='black')
axes[1, 0].plot(df_advanced.index, df_advanced['upper_band'], label='Upper Band', alpha=0.8, color='red')
axes[1, 0].plot(df_advanced.index, df_advanced['lower_band'], label='Lower Band', alpha=0.8, color='green')

# Highlight band walks
upper_walks = df_advanced[df_advanced['upper_band_walk']]
lower_walks = df_advanced[df_advanced['lower_band_walk']]

if len(upper_walks) > 0:
    axes[1, 0].scatter(upper_walks.index, upper_walks['close'], 
                     color='red', s=50, marker='o', label='Upper Band Walk', alpha=0.7)
if len(lower_walks) > 0:
    axes[1, 0].scatter(lower_walks.index, lower_walks['close'], 
                     color='green', s=50, marker='o', label='Lower Band Walk', alpha=0.7)

axes[1, 0].set_title('Bollinger Band Walks')
axes[1, 0].set_ylabel('Price (USDT)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# %B and Price Correlation
axes[1, 1].plot(df_advanced.index, df_advanced['price_pb_corr'], label='Price-%B Correlation', color='blue')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Positive Correlation')
axes[1, 1].axhline(y=-0.5, color='red', linestyle='--', alpha=0.5, label='Negative Correlation')
axes[1, 1].set_title('Price-%B Correlation')
axes[1, 1].set_ylabel('Correlation')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(-1, 1)

plt.tight_layout()
plt.show()

# Analysis of band walks
print("Band Walk Analysis:")
print(f"Upper band walks detected: {df_advanced['upper_band_walk'].sum()}")
print(f"Lower band walks detected: {df_advanced['lower_band_walk'].sum()}")
print(f"Average bandwidth during upper walks: {df_advanced[df_advanced['upper_band_walk']]['bandwidth'].mean():.3f}")
print(f"Average bandwidth during lower walks: {df_advanced[df_advanced['lower_band_walk']]['bandwidth'].mean():.3f}")

## 13. Risk Management with Bollinger Bands

### 1. Position Sizing Based on Bandwidth
- **High Volatility**: Reduce position size when bandwidth is high
- **Low Volatility**: Increase position size when bandwidth is low
- **Squeeze Setup**: Larger positions during squeeze conditions

### 2. Stop Loss Placement
- **Mean Reversion**: Place stop loss beyond the opposite band
- **Trend Following**: Place stop loss below/above the middle band
- **Breakout**: Place stop loss at the edge of the squeeze pattern

### 3. Take Profit Levels
- **Band Targets**: Take profit when price reaches opposite band
- **%B Targets**: Take profit when %B reaches extreme levels
- **Scaling Out**: Partial profits at different band levels

In [ ]:
# Risk Management with Bollinger Bands
df_risk = df_manual.copy()

# Calculate ATR for volatility-based stops
df_risk['atr'] = IndicatorCalculator.atr(df_risk['high'], df_risk['low'], df_risk['close'], 14)

# Risk management techniques
df_risk['percent_b'] = df_manual['percent_b']
df_risk['bandwidth'] = df_manual['bandwidth']
df_risk['upper_band'] = df_manual['upper_band']
df_risk['lower_band'] = df_manual['lower_band']

# 1. Position sizing based on bandwidth
avg_bandwidth = df_risk['bandwidth'].mean()
df_risk['position_size'] = np.where(
    df_risk['bandwidth'] > avg_bandwidth * 1.5, 0.5,  # Small position in high volatility
    np.where(
        df_risk['bandwidth'] < avg_bandwidth * 0.7, 1.0,  # Large position in low volatility
        0.75  # Normal position
    )
)

# 2. Stop loss based on Bollinger Bands and ATR
stop_loss_multiplier = 1.5
df_risk['stop_loss_long'] = df_risk['close'] - (stop_loss_multiplier * df_risk['atr'])
df_risk['stop_loss_short'] = df_risk['close'] + (stop_loss_multiplier * df_risk['atr'])

# 3. Risk-reward ratio based on %B distance
df_risk['pb_distance_from_center'] = abs(df_risk['percent_b'] - 0.5)
df_risk['reward_per_unit'] = df_risk['pb_distance_from_center'] * 0.5  # Scale factor
df_risk['risk_per_unit'] = df_risk['atr'] * stop_loss_multiplier
df_risk['risk_reward_ratio'] = df_risk['reward_per_unit'] / df_risk['risk_per_unit']

# Plot risk management
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position sizing
axes[0, 0].plot(df_risk.index, df_risk['position_size'], label='Position Size', color='blue', linewidth=2)
axes[0, 0].fill_between(df_risk.index, df_risk['position_size'], alpha=0.3, color='blue')
axes[0, 0].axhline(y=avg_bandwidth, color='red', linestyle='--', alpha=0.5, label='Avg Bandwidth')
axes[0, 0].set_title('Position Sizing Based on Bandwidth')
axes[0, 0].set_ylabel('Position Size (Multiplier)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Bandwidth and position size
axes[0, 1].scatter(df_risk['bandwidth'], df_risk['position_size'], alpha=0.6, c=df_risk['bandwidth'], cmap='RdYlGn')
axes[0, 1].axvline(x=avg_bandwidth, color='red', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=avg_bandwidth * 1.5, color='red', linestyle=':', alpha=0.5, label='High Volatility')
axes[0, 1].axvline(x=avg_bandwidth * 0.7, color='green', linestyle=':', alpha=0.5, label='Low Volatility')
axes[0, 1].set_title('Position Size vs Bandwidth')
axes[0, 1].set_xlabel('Bandwidth')
axes[0, 1].set_ylabel('Position Size')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Risk-reward ratio
axes[1, 0].plot(df_risk.index, df_risk['risk_reward_ratio'], label='Risk-Reward Ratio', color='orange')
axes[1, 0].axhline(y=2, color='red', linestyle='--', label='Minimum 2:1 Ratio')
axes[1, 0].fill_between(df_risk.index, df_risk['risk_reward_ratio'], 2, alpha=0.3, color='orange')
axes[1, 0].set_title('Risk-Reward Ratio Analysis')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Bandwidth distribution
axes[1, 1].hist(df_risk['bandwidth'].dropna(), bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[1, 1].axvline(x=avg_bandwidth, color='red', linestyle='--', label='Average Bandwidth')
axes[1, 1].axvline(x=avg_bandwidth * 1.5, color='red', linestyle=':', label='High Volatility Threshold')
axes[1, 1].axvline(x=avg_bandwidth * 0.7, color='green', linestyle=':', label='Low Volatility Threshold')
axes[1, 1].set_title('Bandwidth Distribution')
axes[1, 1].set_xlabel('Bandwidth')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risk management statistics
print("Risk Management Statistics:")
print(f"Average position size: {df_risk['position_size'].mean():.2f}")
print(f"Average risk-reward ratio: {df_risk['risk_reward_ratio'].mean():.2f}")
print(f"Trades with minimum 2:1 ratio: {(df_risk['risk_reward_ratio'] >= 2).sum()} ({(df_risk['risk_reward_ratio'] >= 2).mean()*100:.1f}%)")
print(f"Average bandwidth: {df_risk['bandwidth'].mean():.3f}")
print(f"Bandwidth range: {df_risk['bandwidth'].min():.3f} - {df_risk['bandwidth'].max():.3f}")

## 14. Conclusion

Bollinger Bands are a powerful volatility-based indicator that provides valuable insights into market conditions. While they have limitations, particularly in strong trending markets, they remain one of the most reliable indicators for identifying volatility, overbought/oversold conditions, and potential breakouts.

### Key Takeaways:
1. **Volatility Measurement**: Bollinger Bands excel at measuring market volatility
2. **Dynamic Levels**: Bands automatically adjust to market conditions
3. **Squeeze Detection**: Bandwidth can identify low volatility periods that precede breakouts
4. **Mean Reversion**: Effective for price returning to the mean
5. **Combination**: Works best when combined with other indicators

### Best Practices:
- Use multiple Bollinger Bands parameters for confirmation
- Combine with momentum indicators like RSI or MACD
- Implement proper risk management techniques
- Consider market conditions when interpreting Bollinger Bands signals
- Test different parameters to find what works best for your trading style

## 15. Exercises

### Exercise 1: Optimize Bollinger Bands Parameters
Test different Bollinger Bands parameters (period, standard deviations) to find the optimal combination for different market conditions. Create a function that tests multiple parameter combinations and returns the best performing one based on Sharpe ratio.

### Exercise 2: Create Bollinger Bands Strategy with Volume Confirmation
Modify the Bollinger Bands mean reversion strategy to include volume confirmation. Only enter trades when volume is above the 20-day average and price signals are confirmed.

### Exercise 3: Implement Bollinger Bands with Multiple Timeframe Analysis
Create a strategy that uses Bollinger Bands signals from multiple timeframes (e.g., daily, 4-hour, 1-hour) and only enters trades when signals align across timeframes.

### Solutions:
```python
# Exercise 1 Solution
def optimize_bb_parameters(data, period_range, std_range):
    best_sharpe = -float('inf')
    best_params = None
    
    for period in period_range:
        for std_dev in std_range:
            signals = bb_mean_reversion_strategy(data, period, std_dev)[['signal']]
            engine = BacktestEngine()
            result = engine.run_backtest(data, signals, f"BB_{period}_{std_dev}")
            sharpe = result['metrics']['sharpe_ratio']
            
            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_params = (period, std_dev)
    
    return best_params, best_sharpe

# Exercise 2 Solution
def bb_volume_strategy(data, period=20, std_dev=2, volume_period=20):
    df = data.copy()
    
    # Calculate Bollinger Bands and volume SMA
    df['middle_band'], df['upper_band'], df['lower_band'] = IndicatorCalculator.bollinger_bands(
        df['close'], period, std_dev
    )
    df['volume_sma'] = IndicatorCalculator.sma(df['volume'], volume_period)
    
    # Generate signals with volume confirmation
    df['signal'] = 0
    
    # Buy signal: Price below lower band AND high volume
    buy_condition = ((df['close'] <= df['lower_band']) & 
                    (df['volume'] > df['volume_sma']))
    
    # Sell signal: Price above upper band AND high volume
    sell_condition = ((df['close'] >= df['upper_band']) & 
                      (df['volume'] > df['volume_sma']))
    
    df.loc[buy_condition, 'signal'] = 1
    df.loc[sell_condition, 'signal'] = -1
    
    return df
```

## 16. Additional Resources

- **Books**:
  - "Bollinger on Bollinger Bands" by John Bollinger
  - "Technical Analysis of Financial Markets" by John J. Murphy
  - "Trading Systems and Methods" by Perry J. Kaufman

- **Online Resources**:
  - [Bollinger Bands Official Website](https://www.bollingerbands.com/)
  - [TradingView Documentation](https://www.tradingview.com/pine-script-docs/)
  - [Investopedia - Bollinger Bands](https://www.investopedia.com/terms/b/bollingerbands.asp)
  - [Binance Academy - Technical Analysis](https://academy.binance.com/en/categories/technical-analysis)

- **Research Papers**:
  - "The Profitability of Technical Analysis in the Foreign Exchange Market" by Cheol-Ho Park and Scott H. Irwin
  - "Technical Analysis and Volatility Strategies in Commodity Futures" by Szakmary and Shen

- **Tools**:
  - [TradingView](https://www.tradingview.com/) - Charting and Pine Script
  - [Binance](https://www.binance.com/) - Trading and indicators
  - [Freqtrade](https://www.freqtrade.io/) - Algorithmic trading

---

**Disclaimer**: This tutorial is for educational purposes only. Trading cryptocurrencies involves significant risk. Always do your own research and consider consulting with a financial advisor before making investment decisions.